In [1]:
#Complete cell to create money metrics df for merging
#importing stuff
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline

#importing data from csz
tn_movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
tn_movie_budgets_df

#renaming it to work on
money_metrics_df = tn_movie_budgets_df

#funtion for cleaning
def clean_a_money_column(df, series_title):
    df[series_title] = df[series_title].str.replace('$','')
    df[series_title] = df[series_title].str.replace(',','')
    df[series_title] = pd.to_numeric(df[series_title])
    return df

#cleaning
money_metrics_df = clean_a_money_column(money_metrics_df,'worldwide_gross')
money_metrics_df = clean_a_money_column(money_metrics_df,'domestic_gross')
money_metrics_df = clean_a_money_column(money_metrics_df,'production_budget')

#adding profit margin and ROI columns
money_metrics_df['Profit Margin']= (money_metrics_df['worldwide_gross']-money_metrics_df['production_budget'])/money_metrics_df['worldwide_gross']
money_metrics_df['ROI'] = (money_metrics_df['worldwide_gross']/money_metrics_df['production_budget'])

#setting profit margin desc as order for the table
money_metrics_df=money_metrics_df.sort_values(['Profit Margin'], ascending=False)

#creating year and month columns
money_metrics_df['Year'] = money_metrics_df['release_date']


#Vu's magic lambda, plus to numeric on the year 
money_metrics_df['Year'] = pd.to_numeric(money_metrics_df['release_date'].map(lambda x: x[-4:]))
money_metrics_df['Month'] = money_metrics_df['release_date'].map(lambda x: x[:3])

#functions to pull years, months out of the data set. Can also be used for other column types?
#for now, use int types for the year
def year_iso (df, year):
    output_df = df[df['Year'] == year]
    return output_df

#for now, with money_metrics_df, use three letter abreviations for months
def month_iso (df, month):
    output_df = df[df['Month'] == month]
    return output_df


#Automating year pull outs on money_metrics_df, not sure the best way to do this, but for now, list of dataframes?
year_range = list(range(1915,2021))
year_range
year_keys = []
for i in year_range:
    keystring = ''
    keystring = str(i) + ' money_metric_df'
    year_keys.append(keystring)

list_of_dfs_years = []
for i in year_range:
    df_out = year_iso (money_metrics_df, i)
    list_of_dfs_years.append(df_out)
    
years_and_dfs = dict(zip(year_keys,list_of_dfs_years))



money_metrics_df

#dropping the zeros

money_metrics_df = money_metrics_df[money_metrics_df['worldwide_gross'] != 0]

In [2]:
money_metrics_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,Profit Margin,ROI,Year,Month
5745,46,"Jun 30, 1972",Deep Throat,25000,45000000,45000000,0.999444,1800.000000,1972,Jun
5613,14,"Mar 21, 1980",Mad Max,200000,8750000,99750000,0.997995,498.750000,1980,Mar
5492,93,"Sep 25, 2009",Paranormal Activity,450000,107918810,194183034,0.997683,431.517853,2009,Sep
5679,80,"Jul 10, 2015",The Gallows,100000,22764410,41656474,0.997599,416.564740,2015,Jul
5406,7,"Jul 14, 1999",The Blair Witch Project,600000,140539099,248300000,0.997584,413.833333,1999,Jul
...,...,...,...,...,...,...,...,...,...,...
5335,36,"Nov 21, 2014",Food Chains,913000,0,176,-5186.500000,0.000193,2014,Nov
2152,53,"Aug 24, 1997",The Grimm Brothers' Snow White,26000000,5000,5000,-5199.000000,0.000192,1997,Aug
3818,19,"May 8, 2015",Skin Trade,9000000,1242,1242,-7245.376812,0.000138,2015,May
4081,82,"May 21, 2010",Perrierâs Bounty,6600000,828,828,-7970.014493,0.000125,2010,May
